In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import lit, concat, col, split
from influxdb import InfluxDBClient
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS

In [2]:
class InfluxClient:
    def __init__(self,token,org,bucket):
        self._org=org
        self._bucket = bucket
        self._client = InfluxDBClient(url="http://localhost:8086", token=token)

    def write_data(self,data,write_option=SYNCHRONOUS):
        write_api = self._client.write_api(write_option)
        write_api.write(self._bucket, self._org , data,write_precision='s')

In [3]:
token = 'URJc3EOGIeMEttzZESUtAkMxKq7z4D89q0Yf8J38_7J0UI8Z0XLPhqkKJ-Y7uMf1AbpoVmNDjC8hHVAd0dF7Rg=='
org = 'Scaler'
bucket = 'crypto'
IC = InfluxClient(token,org,bucket)

In [4]:
def save_row(r):
        IC.write_data(r)

In [5]:
scala_version = '2.12'
spark_version = '3.3.2'
# TODO: Ensure match above values match the correct versions
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.4.0'
]

#import os

#spark_version = '3.3.2'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{}'.format(spark_version)

spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()

23/06/24 02:27:13 WARN Utils: Your hostname, Amits-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
23/06/24 02:27:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/amitsingh/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/amitsingh/.ivy2/cache
The jars for the packages stored in: /Users/amitsingh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cd0e9730-724c-4909-9c34-17f3d6a6f211;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.4.

23/06/24 02:27:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
kafka_raw_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "cryptos") \
  .option("startingOffsets", "earliest") \
  .option("key.deserializer","org.common.serialization.StringDeserializer") \
  .option("value.deserializer","org.common.serialization.StringDeserializer") \
  .load()


# Send data in format "measurement BTC-USD=19890.1212"
crypto_df = kafka_raw_df.select(concat(lit("cryptos "), split(col("value"),",")[0], lit("="), split(col("value"),",")[1]))
query = crypto_df.writeStream.foreach(save_row).trigger(processingTime='60 seconds').start()
query.awaitTermination()

23/06/24 02:27:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/cv/c3d_91nj05vg4w2lb_tg1l200000gn/T/temporary-a623b9b2-995a-47a8-9ec4-66ba84e155a6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/24 02:27:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/06/24 02:27:22 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:27:22 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-

23/06/24 02:27:33 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:27:33 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:27:34 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:27:34 WARN NetworkClient:

23/06/24 02:27:47 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:27:47 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:27:48 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:27:48 WARN NetworkClient:

23/06/24 02:28:02 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:28:02 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:28:03 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:28:03 WARN NetworkClient:

23/06/24 02:28:16 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:28:16 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:28:17 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:28:17 WARN NetworkClient:

...

23/06/24 02:36:52 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:36:52 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:36:53 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:36:53 WARN NetworkClient:

23/06/24 02:37:06 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:37:06 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
23/06/24 02:37:07 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:37:07 WARN NetworkClient:

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/amitsingh/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/amitsingh/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/amitsingh/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


23/06/24 02:37:20 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/06/24 02:37:20 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0-1, groupId=spark-kafka-source-9763db15-3779-4725-a866-b30bade5bf43-1999917323-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected


KeyboardInterrupt: 

In [ ]:
ß